Let's do a simple neuron only with Python:

In [70]:
import math

inputs = [.5, .3, .2]
ws= [.4, .7, .2]

#basically this just calculates the Sigmoid fuction
def sigmoid(x):
    y= 1.0 / (1 + math.exp(-x))
    return y

#This is the activation of the neuron
def activate(inputs,ws):
    sum=0
    for input,w in zip(inputs,ws):
        sum += input*w
    return sigmoid(sum)

output= activate(inputs, ws)
print(output)

0.610639233949222


Let's use the dot notation using numpy and also using the sigmoid function directly from tensorflow:

In [71]:
import numpy as np
import tensorflow as tf

inputs = [.5, .3, .2]
ws= [.4, .7, .2]

#this does the dot notation using numpy
# and multiplies the vectors directly
sum=np.dot(inputs,ws)

#we can just directly use the sigmoid function
#from tensorflow
output=tf.sigmoid(sum)

print(output)

tf.Tensor(0.6106392339492219, shape=(), dtype=float64)


Now let's build a neural network (a multi layer perception model) only with Python and NumPy:

In [72]:
class MLP:
    def __init__(self, num_inputs = 3, num_hidden=[3, 5], num_outputs=2):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        layers = [num_inputs] + num_hidden + [num_outputs]

        self.weights = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            self.weights.append(w)


    def forward_propogate(self,inputs):
        activations = inputs

        for w in self.weights:
            net_inputs = np.dot(activations, w)

            activations = self.sigmoid(net_inputs)
        return activations


    def sigmoid(self, x):
        y = 1.0 / (1 + np.exp(-x))
        return y

So no that we made the NN object, lets create one MLP, give it some inputs, run the prop function and print the result:

In [73]:
mlp = MLP()
inputs = np.random.rand(mlp.num_inputs)
outputs = mlp.forward_propogate(inputs)

print("The network input is: {}".format(inputs))
print("The network output is: {}".format(outputs))

The network input is: [0.75706851 0.82002174 0.44752078]
The network output is: [0.8967814  0.75884141]


Let's get into TensorFlow a little bit more.

In [74]:
import tensorflow as tf
#print("TensorFlow version:", tf.__version__)

#let's load the MNIST dataset and scale it to fit between 0 and 1
#The MNIST dataset is a large database of handwritten digits
# that is commonly used for training and testing
# in the field of machine learning.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [75]:
#let's build the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()
predictions





array([[-0.780684  , -0.6758679 ,  0.68717647, -0.28426346, -0.06834835,
         0.06010759, -0.02919856,  0.06456028, -0.4606641 ,  0.5630052 ]],
      dtype=float32)

In [76]:
tf.nn.softmax(predictions).numpy()

array([[0.04522997, 0.05022817, 0.19629535, 0.07430516, 0.09221249,
        0.10485218, 0.0958942 , 0.10532009, 0.06228869, 0.17337365]],
      dtype=float32)

In [77]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1], predictions).numpy()

2.2552037

In [78]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [91]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 2s 961us/step - loss: 0.0638 - accuracy: 0.9797
Epoch 2/10
1875/1875 [==============================] - 2s 860us/step - loss: 0.0559 - accuracy: 0.9824
Epoch 3/10
1875/1875 [==============================] - 2s 918us/step - loss: 0.0526 - accuracy: 0.9831
Epoch 4/10
1875/1875 [==============================] - 2s 932us/step - loss: 0.0462 - accuracy: 0.9848
Epoch 5/10
1875/1875 [==============================] - 2s 996us/step - loss: 0.0443 - accuracy: 0.9855
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0383 - accuracy: 0.9869
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0379 - accuracy: 0.9873
Epoch 8/10
1875/1875 [==============================] - 2s 949us/step - loss: 0.0356 - accuracy: 0.9883
Epoch 9/10
1875/1875 [==============================] - 2s 945us/step - loss: 0.0342 - accuracy: 0.9885
Epoch 10/10
1875/1875 [==============================] - 2s 1ms/step

In [92]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0771 - accuracy: 0.9803 - 226ms/epoch - 722us/step


[0.07712212204933167, 0.9803000092506409]

In [104]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

predictions = probability_model(x_test[:2])
predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)

[7 2]
